# CREAZIONE DEL GRAFO
- Per poter risolvere la seconda parte del progetto abbiamo creato un grafo diretto che collegasse ogni città alle sue tre città più vicine (con longitudine inferiore)

In [1]:
# Importo i pacchetti utili per poter creare il grafo
import pandas as pd
import geopandas
import numpy as np
from geopy import distance
from geopy.distance import geodesic
import networkx as nx
from pyvis.network import Network
import pickle

In [2]:
# importo il dataset pulito ed aggiornato
df= pd.read_csv('Data/dataset_cities_finale_2020.csv')
df

,Unnamed: 0,dt,AverageTemperature,City,Country,Latitude,Longitude,CLL
0,0,1894-01-01,7.475,A Coruña,Spain,42.59,-8.73,A Coruña 42.59 -8.73
1,1,1894-01-01,-0.244,Aachen,Germany,50.63,6.34,Aachen 50.63 6.34
2,2,1894-01-01,0.250,Aalborg,Denmark,57.05,10.33,Aalborg 57.05 10.33
3,3,1894-01-01,25.866,Aba,Nigeria,5.63,8.07,Aba 5.63 8.07
4,4,1894-01-01,11.048,Abadan,Iran,29.74,48.00,Abadan 29.74 48.0
...,...,...,...,...,...,...,...,...
5349235,5349235,2020-12-01,5.009,Århus,Denmark,57.05,10.33,Århus 57.05 10.33
5349236,5349236,2020-12-01,9.843,Çorlu,Turkey,40.99,27.69,Çorlu 40.99 27.69
5349237,5349237,2020-12-01,4.694,Çorum,Turkey,40.99,34.08,Çorum 40.99 34.08
5349238,5349238,2020-12-01,-13.483,Öskemen,Kazakhstan,50.63,82.39,Öskemen 50.63 82.39


In [3]:
# creazione di un dataframe di tutte le 3510 città
df_cities= df[:3510][['CLL', 'Latitude','Longitude']]

In [4]:
# creazione lista del codice univoco delle città
cll=[]
for i in range(df_cities.shape[0]):
    cll.append(df.loc[i]['CLL'])

In [5]:
# creazione di un dizionario con codice univoco di ogni città come chiave e le coordinate come valori

coord_dict={}

for city in cll:
    
    coord_dict[city]=(float(city.split()[-2]),float(city.split()[-1]))


In [6]:
# funzione per calcolare la distanza geodesica tra due città
geodist= lambda x, y: geodesic(coord_dict[x],coord_dict[y]).km

In [7]:
# funzione per trovare in dataframe delle città che si trovano a sinistra della città selezionata
def df_cll_sx(a):
    iniz=coord_dict[a]
    s= df_cities[(df_cities['Longitude']<iniz[1])] 
    return(s)

In [8]:
# funzione per ricavare una lista delle 3 città più vicine della città selezionata
def nearest_city(a):
    try:
    
        distanze= []
        df_city=df_cll_sx(a)

        for c in df_city['CLL']:
            d = geodist(c,a)
            distanze.append(d)

        df_city['distances']=distanze

        df_city=df_city.sort_values(by=['distances'], ascending=True).reset_index()

        n_list=[]

        for j in range(3):

            n_list.append([df_city.loc[j]['Latitude'],df_city.loc[j]['Longitude'],df_city.loc[j]['CLL'],df_city.loc[j]['distances']])

    except KeyError:
        
        n_list=[]
            
    return(n_list)

In [9]:
# creazione di un grafo diretto vuoto 
G= nx.DiGraph()

# aggiunta delle città come nodi del grafo
for city in cll: 
    G.add_node(city)
    
# costruzione degli archi del grafo
for city in cll:
    
    l =nearest_city(city)
    
    if l==[]:
        continue
    else:
        G.add_edge(city,l[0][2])
        G.add_edge(city,l[1][2])
        G.add_edge(city,l[2][2])

C:\Users\lucam\AppData\Local\Temp/ipykernel_12956/2855832825.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_city['distances']=distanze


In [13]:
# visualizzazione degli archi del grafo di una città esempio
G.edges('A Coruña 42.59 -8.73')

OutEdgeDataView([('A Coruña 42.59 -8.73', 'Agadir 29.74 -9.23'), ('A Coruña 42.59 -8.73', 'Las Palmas 28.13 -15.45'), ('A Coruña 42.59 -8.73', 'Santa Cruz De Tenerife 28.13 -15.45')])

In [ ]:
# salvataggio del grafo
nx.readwrite.gpickle.write_gpickle(G,'Data/grafo1')

In [ ]:
#visualizzazione del grafo diretto
net= Network(notebook=True)
net.from_nx(G)
net.show('g1.html')